<img src="img/python-logo-notext.svg"
     style="display:block;margin:auto;width:10%"/>
<br>
<div style="text-align:center; font-size:200%;"><b>Externe Programme</b></div>
<br/>
<div style="text-align:center;">Dr. Matthias Hölzl</div>

## Sub-Prozesse

*Hinweis:* Zur Ausführung dieses Notebooks müssen muss das `ext_sample_app`
Package (in `Examples/ExternalSampleApplication`) installiert sein.

`subprocess.run` ist die bevorzugte Methode um externe Applikationen zu starten.

In [40]:
from subprocess import TimeoutExpired, run

In [41]:
# This may not work if `python` is not in your path...
run(["python", "--version"])

CompletedProcess(args=['python', '--version'], returncode=0)

Mit `shutil.which()` kann man den vollständigen Pfad eines Programms herausfinden.

In [44]:
import shutil

shutil.which("python")

'C:\\Users\\tc\\Programming\\Python\\Anaconda3\\envs\\cam\\python.EXE'

In [45]:
cp = run([shutil.which("python"), "--version"])

In [46]:
def print_completed_process(cp):
    print("return code:", cp.returncode)
    print("captured stdout:", repr(cp.stdout))
    print("captured stderr:", repr(cp.stderr))

In [47]:
print_completed_process(cp)

return code: 0
captured stdout: None
captured stderr: None


In [48]:
cp = run([shutil.which("python"), "--version"], capture_output=True, text=True)

In [49]:
print_completed_process(cp)

return code: 0
captured stdout: 'Python 3.10.4\n'
captured stderr: ''


Mit `sys.executable` kann man den Pfad des gerade aktiven Python Interpreters herausfinden. Das ist die bevorzugte Methode um einen Python Prozess zu starten.

In [50]:
import sys
sys.executable

'C:\\Users\\tc\\Programming\\Python\\Anaconda3\\envs\\cam\\python.exe'

In [51]:
cp = run([sys.executable, "--version"], capture_output=True, text=True)

In [52]:
print_completed_process(cp)

return code: 0
captured stdout: 'Python 3.10.4\n'
captured stderr: ''


In [53]:
cp = run([sys.executable, "-m", "ext_sample_app"], capture_output=True, text=True)

In [54]:
print_completed_process(cp)

return code: 2
captured stdout: ''
captured stderr: "Usage: python -m ext_sample_app [OPTIONS] COMMAND [ARGS]...\nTry 'python -m ext_sample_app --help' for help.\n\nError: Missing command.\n"


In [55]:
cp = run(
    [sys.executable, "-m", "ext_sample_app", "--help"], capture_output=True, text=True
)

In [56]:
print_completed_process(cp)

return code: 0
captured stdout: 'Usage: python -m ext_sample_app [OPTIONS] COMMAND [ARGS]...\n\nOptions:\n  --help  Show this message and exit.\n\nCommands:\n  error\n  interact\n  say-hi\n  serve\n'
captured stderr: ''


In [57]:
cp = run(
    [sys.executable, "-m", "ext_sample_app", "say-hi"], capture_output=True, text=True
)

In [58]:
print_completed_process(cp)

return code: 0
captured stdout: 'Hello, world!\n'
captured stderr: ''


In [59]:
cp = run(
    [sys.executable, "-m", "ext_sample_app", "error"], capture_output=True, text=True
)

In [61]:
print_completed_process(cp)

return code: 1
captured stdout: ''
captured stderr: 'An error occurred!\n'


In [62]:
dir(cp)

['__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'args',
 'check_returncode',
 'returncode',
 'stderr',
 'stdout']

In [ ]:
# THIS DOES NOT WORK!
# cp = run(
#     [sys.executable, "-m", "ext_sample_app", "interact"], capture_output=True, text=True
# )

## Popen: Nebenläufige Ausführung von Programmen

Wenn man nicht warten kann, bis das gestartete Programm beendet wird muss man die `subprocess.Popen` Klasse verwenden:

In [63]:
from subprocess import Popen, PIPE
import sys

In [64]:
proc = Popen(
    [sys.executable, "-m", "ext_sample_app", "interact"],
    stdin=PIPE,
    stderr=PIPE,
    stdout=PIPE,
    encoding="utf-8",
    universal_newlines=True,
    bufsize=0,
)

In [65]:
type(proc)

subprocess.Popen

`proc.communicate()` sendet eine Nachricht and `proc`, schließt die Ein- und Ausgabeströme und beendet den Prozess.

In [66]:
proc.communicate("work")

('Please enter a command: Working...done.\n', '')

Mit `proc.poll()` kann man feststellen, ob der Prozess schon beendet wurde und was der Rückgabewert war. Falls das Ergebniss `None` ist, ist der Prozess noch aktiv. `proc.wait()` wartet eine bestimmte Zeit und gibt den Rückgabewert des Prozesses zurück. Falls der Prozess nicht in der vorgegebenen Zeit beendet wurde, wird eine `TimeoutExpired` Exception ausgelöst.

In [67]:
proc.poll()

0

In [68]:
def run_and_communicate(command):
    proc = Popen(
        [sys.executable, "-m", "ext_sample_app", "interact"],
        stdin=PIPE,
        stderr=PIPE,
        stdout=PIPE,
        encoding="utf-8",
        universal_newlines=True,
        bufsize=0,
    )
    result = proc.communicate(command)
    try:
        wait_result = proc.wait(5)
    except TimeoutExpired:
        print("Process did not terminate!")
        proc.terminate()
        wait_result = proc.wait(5)
    return result, wait_result

In [69]:
run_and_communicate("work")

(('Please enter a command: Working...done.\n', ''), 0)

In [70]:
run_and_communicate("exit")

(('Please enter a command: Exiting!\n', ''), 0)

In [71]:
run_and_communicate("error")

(('Please enter a command: ', "ERROR: Illegal command 'error'!\n"), 2)

## Kommunikation mit Sockets

Das folgende Beispiel zeigt, wie man einen Prozess starten und dann über Sockets mit ihm kommunizieren kann.

In [72]:
from subprocess import Popen, PIPE
import sys

HOST = "localhost"
PORT = 12345

In [73]:
from socket import socket, AF_INET, SOCK_STREAM
import sys


def send_message(msg: str):
    with socket(AF_INET, SOCK_STREAM) as sock:
        sock.connect((HOST, PORT))
        sock.sendall(bytes(msg + "\n", "utf-8"))
        return str(sock.recv(1024), "utf-8")

In [74]:
proc = Popen(
    [
        sys.executable,
        "-m",
        "ext_sample_app",
        "serve",
        "--host",
        HOST,
        "--port",
        str(PORT),
    ],
    stdin=PIPE,
    stderr=PIPE,
    stdout=PIPE,
    encoding="utf-8",
    universal_newlines=True,
    bufsize=0,
)

In [76]:
proc.poll() is None

True

In [79]:
send_message("Hello, world!")

'HELLO, WORLD!'

In [80]:
send_message("Are you running?")

'ARE YOU RUNNING?'

In [82]:
proc.poll() is None

True

In [83]:
proc.terminate()

In [84]:
proc.poll()

1

In [85]:
try:
    send_message("Are you still running?")
except ConnectionRefusedError as err:
    print("Could not connect to server.")
    print(err)

Could not connect to server.
[WinError 10061] No connection could be made because the target machine actively refused it


In [86]:
proc.poll()

1